In [2]:
import pandas as pd
df = pd.read_csv('https://media.githubusercontent.com/media/metmuseum/openaccess/master/MetObjects.csv')

C:\Users\brian\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7,8,9,10,11,18,27,28,29,30,31,32,33,34,35,36,37,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def getID_URLvalidlist(filename='final_data.txt'):     
    data_list = []
    result_list = []
    with open(filename, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            data_list.append(line.strip())
            temp_list = data_list[0].split('), (')
    for i in temp_list:
        i = i.replace('[', '')
        i = i.replace('(', '')
        i = i.replace(']', '')
        i = i.replace(')', '')
        i = i.replace(" '" , '')
        i = i.replace("'", '')
        i = i.split(',', 1)
        i = [int(i[0]),i[1]]
        result_list.append(i)
    return result_list

In [4]:
import numpy as np

valid_URL_list = getID_URLvalidlist()

def merge_image_df(df, url_list):
    image_object_id = []
    image_urls = []
    for x in url_list:
        image_object_id.append(x[0])
        image_urls.append(x[1])

    image_df = pd.DataFrame({'Object ID': image_object_id, 'image_url': image_urls})
    image_df = image_df.set_index('Object ID')

    df = df.set_index('Object ID')
    final_df = df.join(image_df)
    final_df = final_df.reset_index()
    final_df = final_df.rename(columns = {'index': 'Object ID'})
    
    return final_df

In [5]:
df = merge_image_df(df,valid_URL_list)
df=df[['Object ID','Department','Object Name','Title','Artist Display Name','Object Date','Country','Link Resource', 'image_url']]

In [7]:
import re
import string
import numpy as np

def format_replace_words(replace_dict):
    format_replace_dict = dict()
    for k,v in replace_dict.items():
        format_replace_dict[k.lower()] = v
    return format_replace_dict

def multiword_replace(text, replacement_dict):
    pattern = re.compile("|".join([re.escape(k) for k in sorted(replacement_dict,key=len,reverse=True)]), re.I)
    new_text = re.sub(pattern, lambda x: replacement_dict[x.group(0)], text)
    return new_text

def multiregex_replace(text, regex_dict):
    for k,v in regex_dict.items():
        regex = re.compile(k, re.I)
        match = re.search(regex, text)
        if match:
            text = re.sub(regex, v, text)
    return text

def lc_match_group(obj):
    return obj.group().lower()

def formatted_raw_string(text):
    if text is not None:
        text = text.title()
        format_pattern = r"(?<=[a-z])[\']([A-Z])"
        text = re.sub(format_pattern, lc_match_group, text)
    return text

def clean_data(df, category, word_dict, regex_dict, replace_pattern, removal_pattern):
    df[category] = df[category].astype(str)
    df[category] = np.where(df[category].str.contains(replace_pattern, na = False, case=False, regex=True) == True,
                            df[category].str.replace(replace_pattern, '', flags=re.I),
                            (np.where(df[category].str.contains(removal_pattern, na = False, case=False, regex=True) == True,
                                     'nan',
                                     df[category])))
    df[category] = np.where((np.char.find(df[category].str.lower().values.astype(str),"?")!=-1),'nan',df[category])
    
    compiled_list = list(df[category])
    for x in range(len(compiled_list)): 
        compiled_list[x] = " ".join(compiled_list[x].lower().split())
        compiled_list[x] = multiword_replace(compiled_list[x], word_dict)
        compiled_list[x] = multiregex_replace(compiled_list[x], regex_dict)
        compiled_list[x] = formatted_raw_string(compiled_list[x])
        token_list = compiled_list[x].split("|")
        for y in range(len(token_list)):
            token_list[y] = token_list[y].strip()
        token_list = sorted(filter(None, token_list))
        token_list = set(token_list)
        compiled_list[x] = ' | '.join(token_list)
    df[category] = pd.Series(compiled_list)
    df[category] = np.where(df[category] == 'Nan', np.nan, df[category])

In [8]:
def clean_country_data(df):
    
    category = 'Country'
    word_dict = {
        'Algieria': 'Algeria',
        'Republic of Congo': 'Republic Of The Congo',
        'yugoslavia0' : 'Yugoslavia)',
        'Dem. Rep. Congo': 'Democratic Republic Of The Congo',
        '- Turkey': 'Turkey'
    }
    word_dict = format_replace_words(word_dict)
    
    regex_dict = {
        #Inside content text has to be lowercase
        '\s+and\s+': '|',
        '\s*/\s*': '|',
        '^uk$': 'United Kingdom',
        '^u[.]*s[.]*[a.]*': 'United States',
        '\s*&\s*': '|'
    }
    
    replace_pattern = r'[(]*north[)]*\w*|south\w*|west\w*|east\w*|central|nothern|sothern|:\s*n.a.|:n.|harold copping|bernard turner'
    removal_pattern = r'probably|probbly|possibly|perhaps|unknown|africa|asia|europe|america|middle east|mediterranean|lisez|flanders|[^A-Z]+or[^A-Z]+'
    
    clean_data(df, category, word_dict, regex_dict, replace_pattern, removal_pattern)
    

In [9]:
def clean_artist_name_data(df):
    
    category = 'Artist Display Name'
    word_dict = {
        ', and': ',',
        'Benjamin Bakewell & Co.': 'Bakewell, Page & Bakewell',
        'Bakewell, Page & Bakewells': 'Bakewell, Page & Bakewell',
    }
    
    regex_dict = {
        #Inside content text has to be lowercase
        '\s*[/;]\s*': '|',
        '^[A-Z].\s*[A-Z].\s*$': '',
        '^[A-Z].\s*[A-Z].\s*[A-Z].$': '',
        '^A$': '',
    }
    
    word_dict = format_replace_words(word_dict)
    replace_pattern = r'\d+[-]\d+|[.,]{1} paris|[.,]{1} berlin|[.,]{1} new york|[.,]{1} london|[.,]{1} england|"L"'
    removal_pattern = r'probably|prob.|possibly|perhaps|unknown|anonymous|painter|artist|[^A-Z]+or[^A-Z]+|^[A-Z].\s*[A-Z].\s*$|^[A-Z].\s*[A-Z].\s*[A-Z].$'
    
    clean_data(df, category, word_dict, regex_dict, replace_pattern, removal_pattern)

In [10]:
def clean_art_title_data(df):
    
    category = 'Title'
    word_dict = {
        'coffee cup saucer': 'Coffee Cup And Saucer',
        'doorknob': 'Door Knob',
        "\'": "'"
    }
    
    regex_dict = {
        #Inside content text has to be lowercase
        'chest[-]*\s*\w+[-]*\s*drawer[s]*': 'Chest With Drawer',
        '\s*[/]\s*': '/',
    }
    
    word_dict = format_replace_words(word_dict)
    replace_pattern = r'covered|^[[]|[]]$'
    removal_pattern = r'probably|prob.|possibly|perhaps|unknown|anonymous|[^A-Z]+or[^A-Z]+'
    
    clean_data(df, category, word_dict, regex_dict, replace_pattern, removal_pattern)

In [11]:
def clean_object_date(df):
    def extract_non_numeric(object_date):
        string=str(object_date)
        import re
        pattern=r'\D+'
        match=re.findall(pattern,string)
        return tuple(match)
    df['object_date_non_numeric']=df['Object Date'].apply(extract_non_numeric)
    def clean_object_date(date):
        import re
        def extract_year(string):
            pattern=r'\d+'
            m=re.search(pattern,string).group()
            if 3<=len(m)<=4 and int(m)<=2018:
                return int(m[:-2])+1
            elif 1<=len(m)<=2:
                return 1
            else:
                return None
        century=None
        if date==None:
            century=None
        string=str(date)
        if string=='':
            century=None
        match=tuple(re.findall(r'\D+',string))
        if  match==('th century',) or match==('th–','th century') or match==('early ','th century') or match==('late ','th century') or match==('late ','th–early ','th century') or match==('mid-','th century') or match==('second half ','th century') or match==('first half ','th century') or match==('th–early ','th century'):
            pattern=r'\d+'
            m=re.search(pattern,string).group()
            if not m==None:
                century=int(m)
        if match==('th century B.C.',) or match==('ca. ','th–','th century B.C.') or match==('th–','th century B.C.'):
            pattern=r'\d+'
            m=re.search(pattern,string).group()
            if not m==None:
                century=-int(m)
        if match==('ca. ','–',' B.C.') or match==('ca. ',' B.C.') or match==('B.C.–A.D. ',):
            century=extract_year(string)
        if match==tuple() or match==('–',) or match==('s',) or match==('s–','s') or match==('fall/winter ','–') or match==('ca. ','–') or match==('ca. ',) or match==('dated ',) or match==('spring/summer ',):
            century=extract_year(string)
        return century
    df['century']=df['Object Date'].apply(clean_object_date)
    df.drop(['Object Date','object_date_non_numeric'],axis=1,inplace=True)

In [12]:
clean_country_data(df)
clean_artist_name_data(df)
clean_art_title_data(df)
clean_object_date(df)
clean_country_data(df)

In [20]:
df

,Object ID,Department,Object Name,Title,Artist Display Name,Country,Link Resource,image_url,century
0,1,American Decorative Arts,Coin,One-Dollar Liberty Head Coin,James Barton Longacre,NaN,http://www.metmuseum.org/art/collection/search/1,NaN,19.0
1,2,American Decorative Arts,Coin,Ten-Dollar Liberty Head Coin,Christian Gobrecht,NaN,http://www.metmuseum.org/art/collection/search/2,NaN,20.0
2,3,American Decorative Arts,Coin,Two-And-A-Half Dollar Coin,NaN,NaN,http://www.metmuseum.org/art/collection/search/3,NaN,20.0
3,4,American Decorative Arts,Coin,Two-And-A-Half Dollar Coin,NaN,NaN,http://www.metmuseum.org/art/collection/search/4,NaN,20.0
4,5,American Decorative Arts,Coin,Two-And-A-Half Dollar Coin,NaN,NaN,http://www.metmuseum.org/art/collection/search/5,NaN,20.0
5,6,American Decorative Arts,Coin,Two-And-A-Half Dollar Coin,NaN,NaN,http://www.metmuseum.org/art/collection/search/6,NaN,20.0
6,7,American Decorative Arts,Coin,Two-And-A-Half Dollar Coin,NaN,NaN,http://www.metmuseum.org/art/collection/search/7,NaN,20.0
7,8,American Decorative Arts,Coin,Two-And-A-Half Dollar Coin,NaN,NaN,http://www.metmuseum.org/art/collection/search/8,NaN,20.0
8,9,American Decorative Arts,Coin,Two-And-A-Half Dollar Coin,NaN,NaN,http://www.metmuseum.org/art/collection/search/9,NaN,20.0
9,10,American Decorative Arts,Coin,Two-And-A-Half-Dollar Indian Head Coin,Bela Lyon Pratt,NaN,http://www.metmuseum.org/art/collection/search/10,NaN,20.0


In [21]:
df.to_csv('cleaned_data_final.csv', index=False)